## 1. Import Dependencies

In [1]:
conda install -c anaconda swig


Note: you may need to restart the kernel to use updated packages.


Traceback (most recent call last):
  File "C:\Users\danie\anaconda3\Scripts\conda-script.py", line 11, in <module>
    from conda.cli import main
ModuleNotFoundError: No module named 'conda'


In [10]:
!pip install gym[box2d]

In [5]:
!pip install pyglet==1.5.5

  Attempting uninstall: pyglet
    Found existing installation: pyglet 1.5.26
    Uninstalling pyglet-1.5.26:
      Successfully uninstalled pyglet-1.5.26


In [1]:
# Added for video logs
from typing import Any, Dict
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import Video
import torch as th

C:\Users\danie\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\danie\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\danie\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# Add a callback to training stage
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [3]:
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import os

## 2. Test Environment

In [4]:
environment_name = 'CarRacing-v0'
env = gym.make(environment_name)

In [5]:
env.reset()

Track generation: 1229..1541 -> 312-tiles track


array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [6]:
env.action_space

Box([-1.  0.  0.], [1. 1. 1.], (3,), float32)

In [6]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done: 
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Track generation: 1062..1338 -> 276-tiles track
Episode:1 Score:-27.27272727272762
Track generation: 954..1197 -> 243-tiles track
Episode:2 Score:-17.355371900826412
Track generation: 1185..1491 -> 306-tiles track
Episode:3 Score:-34.426229508197274
Track generation: 1108..1397 -> 289-tiles track
Episode:4 Score:-30.555555555556065
Track generation: 1036..1299 -> 263-tiles track
Episode:5 Score:-23.664122137404796


In [5]:
env.close()

## 3. Train Model + Tensorboard Video Logs

video record training

In [59]:
class VideoRecorderCallback(BaseCallback):
    def __init__(self, eval_env: gym.Env, render_freq: int, n_eval_episodes: int = 1, deterministic: bool = True):
        """
        Records a video of an agent's trajectory traversing ``eval_env`` and logs it to TensorBoard

        :param eval_env: A gym environment from which the trajectory is recorded
        :param render_freq: Render the agent's trajectory every eval_freq call of the callback.
        :param n_eval_episodes: Number of episodes to render
        :param deterministic: Whether to use deterministic or stochastic policy
        """
        super().__init__()
        self._eval_env = eval_env
        self._render_freq = render_freq
        self._n_eval_episodes = n_eval_episodes
        self._deterministic = deterministic

    def _on_step(self) -> bool:
        if self.n_calls % self._render_freq == 0:
            screens = []

            def grab_screens(_locals: Dict[str, Any], _globals: Dict[str, Any]) -> None:
                """
                Renders the environment in its current state, recording the screen in the captured `screens` list

                :param _locals: A dictionary containing all local variables of the callback's scope
                :param _globals: A dictionary containing all global variables of the callback's scope
                """
                screen = self._eval_env.render(mode="rgb_array")
                # PyTorch uses CxHxW vs HxWxC gym (and tensorflow) image convention
                screens.append(screen.transpose(2, 0, 1))

            evaluate_policy(
                self.model,
                self._eval_env,
                callback=grab_screens,
                n_eval_episodes=self._n_eval_episodes,
                deterministic=self._deterministic,
            )
            self.logger.record(
                "trajectory/video",
                Video(th.ByteTensor([screens]), fps=40),
                exclude=("stdout", "log", "json", "csv"),
            )
        return True

In [7]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [8]:
log_path = os.path.join('Training', 'Logs')
#model = PPO('CnnPolicy', env, verbose = 1, tensorboard_log = log_path)

In [9]:
# For video callback
video_recorder = VideoRecorderCallback(gym.make(environment_name), render_freq=5000)

NameError: name 'VideoRecorderCallback' is not defined

In [12]:
# Add a callback to training stage
save_path = os.path.join('Training', 'Saved Models')
stop_callback = StopTrainingOnRewardThreshold(reward_threshold = 1000, verbose = 1)
eval_callback = EvalCallback(env, 
                            callback_on_new_best = stop_callback,
                            eval_freq = 5000, 
                            best_model_save_path = save_path, 
                            verbose = 1)

In [10]:
model = PPO('CnnPolicy', env, verbose = 1, tensorboard_log = log_path)


Using cpu device
Wrapping the env in a VecTransposeImage.


In [58]:
model.learn(total_timesteps = 100000, callback = eval_callback)

Track generation: 1220..1529 -> 309-tiles track
Logging to Training\Logs\PPO_33


C:\Users\danie\anaconda3\lib\site-packages\stable_baselines3\common\callbacks.py:345: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.vec_transpose.VecTransposeImage object at 0x000002129B23E190> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x000002128EEA2A60>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


Track generation: 1189..1490 -> 301-tiles track
Track generation: 1114..1396 -> 282-tiles track
Track generation: 1231..1543 -> 312-tiles track
Track generation: 1231..1543 -> 312-tiles track
Track generation: 1184..1484 -> 300-tiles track
Track generation: 1079..1353 -> 274-tiles track
Eval num_timesteps=351, episode_reward=123.86 +/- 55.91
Episode length: 928.80 +/- 102.77
---------------------------------
| eval/              |          |
|    mean_ep_length  | 929      |
|    mean_reward     | 124      |
| time/              |          |
|    total_timesteps | 351      |
---------------------------------
Track generation: 1221..1531 -> 310-tiles track
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.35e+03 |
|    ep_rew_mean     | 513      |
| time/              |          |
|    fps             | 44       |
|    iterations      | 1        |
|    time_elapsed    | 46       |
|    total_timesteps | 2048     |
------------------------------

Track generation: 1223..1533 -> 310-tiles track
Track generation: 1178..1477 -> 299-tiles track
Track generation: 1181..1481 -> 300-tiles track
Track generation: 1145..1435 -> 290-tiles track
Track generation: 1141..1430 -> 289-tiles track
Track generation: 1228..1548 -> 320-tiles track
Track generation: 1312..1644 -> 332-tiles track
Track generation: 1113..1401 -> 288-tiles track
Eval num_timesteps=15351, episode_reward=373.03 +/- 113.18
Episode length: 1000.00 +/- 0.00
---------------------------------------
| eval/                   |           |
|    mean_ep_length       | 1e+03     |
|    mean_reward          | 373       |
| time/                   |           |
|    total_timesteps      | 15351     |
| train/                  |           |
|    approx_kl            | 0.5232031 |
|    clip_fraction        | 0.744     |
|    clip_range           | 0.2       |
|    entropy_loss         | -4.9      |
|    explained_variance   | 0.886     |
|    learning_rate        | 0.0003    |
|   

Track generation: 1165..1463 -> 298-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1152..1444 -> 292-tiles track
Track generation: 1008..1272 -> 264-tiles track
Track generation: 1008..1272 -> 264-tiles track
Track generation: 1111..1393 -> 282-tiles track
Track generation: 1157..1450 -> 293-tiles track
Track generation: 1098..1377 -> 279-tiles track
Track generation: 1145..1435 -> 290-tiles track
Track generation: 961..1208 -> 247-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1171..1468 -> 297-tiles track
Eval num_timesteps=30351, episode_reward=151.56 +/- 143.25
Episode length: 900.80 +/- 198.40
---------------------------------------
| eval/                   |           |
|    mean_ep_length       | 901       |
|    mean_reward          | 152       |
| time/                   |           |
|    total_timesteps      | 30351     |
| train/                  | 

Track generation: 1089..1374 -> 285-tiles track
Track generation: 1151..1442 -> 291-tiles track
Track generation: 1055..1323 -> 268-tiles track
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 928       |
|    ep_rew_mean          | 216       |
| time/                   |           |
|    fps                  | 42        |
|    iterations           | 21        |
|    time_elapsed         | 1022      |
|    total_timesteps      | 43008     |
| train/                  |           |
|    approx_kl            | 0.6292405 |
|    clip_fraction        | 0.747     |
|    clip_range           | 0.2       |
|    entropy_loss         | -5.13     |
|    explained_variance   | 0.875     |
|    learning_rate        | 0.0003    |
|    loss                 | 0.424     |
|    n_updates            | 4910      |
|    policy_gradient_loss | 0.0476    |
|    std                  | 1.35      |
|    value_loss           | 6.85      |
----------------

Track generation: 1087..1363 -> 276-tiles track
Track generation: 1102..1382 -> 280-tiles track
Track generation: 1148..1439 -> 291-tiles track
Track generation: 1131..1418 -> 287-tiles track
Track generation: 1051..1318 -> 267-tiles track
Track generation: 1058..1335 -> 277-tiles track
Eval num_timesteps=55351, episode_reward=143.97 +/- 178.68
Episode length: 876.40 +/- 247.20
---------------------------------------
| eval/                   |           |
|    mean_ep_length       | 876       |
|    mean_reward          | 144       |
| time/                   |           |
|    total_timesteps      | 55351     |
| train/                  |           |
|    approx_kl            | 1.1607779 |
|    clip_fraction        | 0.727     |
|    clip_range           | 0.2       |
|    entropy_loss         | -5.26     |
|    explained_variance   | 0.843     |
|    learning_rate        | 0.0003    |
|    loss                 | 0.382     |
|    n_updates            | 4980      |
|    policy_gradien

Track generation: 1208..1514 -> 306-tiles track
Track generation: 1175..1473 -> 298-tiles track
Track generation: 980..1229 -> 249-tiles track
Track generation: 1052..1327 -> 275-tiles track
Track generation: 1128..1414 -> 286-tiles track
Track generation: 1152..1444 -> 292-tiles track
Eval num_timesteps=70351, episode_reward=217.83 +/- 173.25
Episode length: 936.80 +/- 126.40
---------------------------------------
| eval/                   |           |
|    mean_ep_length       | 937       |
|    mean_reward          | 218       |
| time/                   |           |
|    total_timesteps      | 70351     |
| train/                  |           |
|    approx_kl            | 0.7935419 |
|    clip_fraction        | 0.774     |
|    clip_range           | 0.2       |
|    entropy_loss         | -5.32     |
|    explained_variance   | 0.845     |
|    learning_rate        | 0.0003    |
|    loss                 | 1.8       |
|    n_updates            | 5050      |
|    policy_gradient

Track generation: 1131..1425 -> 294-tiles track
Track generation: 1020..1279 -> 259-tiles track
Track generation: 1139..1428 -> 289-tiles track
Track generation: 1211..1517 -> 306-tiles track
Track generation: 1097..1375 -> 278-tiles track
Track generation: 1080..1353 -> 273-tiles track
Track generation: 997..1253 -> 256-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1252..1569 -> 317-tiles track
Eval num_timesteps=85351, episode_reward=79.36 +/- 164.55
Episode length: 974.80 +/- 50.40
---------------------------------------
| eval/                   |           |
|    mean_ep_length       | 975       |
|    mean_reward          | 79.4      |
| time/                   |           |
|    total_timesteps      | 85351     |
| train/                  |           |
|    approx_kl            | 1.8529673 |
|    clip_fraction        | 0.75      |
|    clip_range           | 0.2       |
|    entropy_loss         | -5.46     |
|    e

Track generation: 1148..1439 -> 291-tiles track
Track generation: 1158..1452 -> 294-tiles track
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 950       |
|    ep_rew_mean          | 179       |
| time/                   |           |
|    fps                  | 41        |
|    iterations           | 48        |
|    time_elapsed         | 2391      |
|    total_timesteps      | 98304     |
| train/                  |           |
|    approx_kl            | 1.2272251 |
|    clip_fraction        | 0.71      |
|    clip_range           | 0.2       |
|    entropy_loss         | -5.5      |
|    explained_variance   | 0.869     |
|    learning_rate        | 0.0003    |
|    loss                 | 0.175     |
|    n_updates            | 5180      |
|    policy_gradient_loss | 0.0469    |
|    std                  | 1.56      |
|    value_loss           | 8.02      |
---------------------------------------
Track generation: 1139..

## 4. Save Model

In [11]:
ppo_path = os.path.join('Training', 'Saved Models', 'PPO_10k_Driving_Model')

In [ ]:
model.save(ppo_path)

In [11]:
del model

In [12]:
ppo_path = os.path.join('Training', 'Saved Models', 'best_model')

In [13]:
model = PPO.load(ppo_path,env)

Wrapping the env in a VecTransposeImage.


## 5. Evaluate

In [14]:
evaluate_policy(model, env, n_eval_episodes = 5, render = True)

C:\Users\danie\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Track generation: 1308..1639 -> 331-tiles track
Track generation: 1094..1373 -> 279-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1112..1394 -> 282-tiles track
Track generation: 1124..1409 -> 285-tiles track
Track generation: 1140..1429 -> 289-tiles track
Track generation: 1188..1492 -> 304-tiles track
Track generation: 1079..1353 -> 274-tiles track


(503.8364300742745, 141.2056876962908)

In [15]:
env.close()

## Test Model

In [35]:
model.predict(obs.copy())

(array([1., 1., 0.], dtype=float32), None)

In [38]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action, _ = model.predict(obs.copy())
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Track generation: 1078..1355 -> 277-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1056..1333 -> 277-tiles track
Episode:1 Score:153.6231884058016
Track generation: 1064..1334 -> 270-tiles track
Episode:2 Score:617.4721189590911
Track generation: 1088..1364 -> 276-tiles track
Episode:3 Score:452.7272727272552
Track generation: 1092..1379 -> 287-tiles track
Episode:4 Score:179.72027972028425
Track generation: 941..1186 -> 245-tiles track
Episode:5 Score:16.637704918034473
